# Classificação Automática de Textos

Até o momento, já conseguimos encontrar a probabilidade de, ao retirarmos um texto aleatório de uma coleção $c$, encontrarmos a palavra $w$, isto é:

$P(w | c)$.

Usando o Teorema de Bayes, podemos encontrar a probabilidade de um texto fazer parte de uma coleção $c$ sabendo que a palavra $w$ foi encontrada, isto é:

$P(c | w) = \frac{P(w | c)P(c)}{P(w)}$.

Nesta aula, aprenderemos a usar essa ideia para classificar textos automaticamente.

In [28]:
import joblib
import urllib
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

frases_positivas = [
    "Que delícia esse sol quente!",
    "Adoro sentir o calor do sol na pele!",
    "Um dia quente é perfeito para curtir uma piscina!",
    "O calor deixa tudo mais animado e colorido!",
    "Nada melhor que um sorvete refrescante em um dia quente!",
    "As roupas leves e soltinhas são a cara do verão!",
    "Adoro o cheiro de protetor solar em um dia quente!",
    "O calor deixa as pessoas mais alegres e descontraídas!",
    "Um dia quente é a desculpa perfeita para tomar uma cerveja gelada!",
    "A sensação de relaxamento que um dia quente traz é incomparável!"
]

frases_negativas = [
    "Este calor está insuportável!",
    "Eu não aguento mais suar o dia inteiro.",
    "Não dá nem vontade de sair de casa com esse sol quente.",
    "O ar-condicionado não dá conta de refrescar o ambiente.",
    "Até o ventilador parece não estar funcionando direito hoje.",
    "Eu odeio dias assim, prefiro o frio mil vezes!",
    "Não tem um lugar com sombra nessa cidade?",
    "Essa roupa colada no corpo me incomoda demais.",
    "Quero chuva, quero frio, quero qualquer coisa, menos esse sol na minha cabeça!",
    "Já acabou o verão? Porque eu não aguento mais esse calor infernal."
]

url = "https://gist.githubusercontent.com/alopes/5358189/raw/2107d809cca6b83ce3d8e04dbd9463283025284f/stopwords.txt"
stopwords_list = urllib.request.urlopen(url).read().decode()
stopwords_ptbr = set(stopwords_list.split())

## Exercício 1
**Objetivo: entender como usar o `.fit` e o `.predict` em um classificador**

Um classificador é um algoritmo que recebe como entrada um vetor de características de algum objeto e retorna a classe à qual esse objeto pertence. Uma das estratégias de classificação que existem é usar o Teorema de Bayes. Para isso, precisamos estimar as distribuições $P(w|c)$, $P(w)$ e $P(c)$.

Quando estamos usando a presença ou não-presença de uma palavra como observação, podemos assumir que isso segue uma distribuição de Bernoulli, ou seja, existe uma probabilidade $p$ de que a palavra exista em um documento e uma probabilidade $1-p$ de que ela não exista, de uma forma semelhante a jogar uma moeda enviesada. O que precisamos, então, é descobrir os viéses dessa "moeda" no conjunto universo ($P(w)$) e dentro de cada uma das classes que nos interessam ($P(w|c_i)$ para toda classe $c_i$).

O processo de descobrir esses viéses se chama *fit*. Para realizá-lo, precisamos fornecer ao classificador:

1. Uma representação dos nossos documentos que indique a presença ou não de cada palavra no documento
1. Uma anotação dizendo a qual classe cada documento pertence

Depois que fazemos isso, podemos testar nosso classificador com um documento desconhecido, e ele retorna a classe mais provável à qual nosso sistema pertence. Esse processo de predição é chamado de *predict*.

Então, teríamos um código como:

In [29]:
X = np.array([ [1, 0, 0, 1], [1, 0, 1, 1], [0, 1, 1, 0], [0, 1, 0, 1]])
y = np.array(['legal', 'legal', 'chato', 'chato'])

classificador = BernoulliNB()
classificador.fit(X, y)
y_pred = classificador.predict(X)
print(y_pred)

y_pred_ = classificador.predict([[1,1,1,1]])
print(y_pred_)

['legal' 'legal' 'chato' 'chato']
['legal']


Vamos treinar como usar essa ideia em nossos dados de frases sobre o calor. Gostaríamos de fazer um sistema que recebe como entrada uma frase e então informa se é uma frase dizendo que gosta ou se nào gosta do calor. Para isso:

1. Una as listas de strings do dataset para encontrar um único dataset;
1. Use o `CountVectorizer` para encontrar as representações vetoriais das frases que estão no nosso dataset; 
1. Crie um vetor `y` com classes correspondentes a gostar ou não gostar do calor;
1. Treine (`.fit()`) um classificador do tipo `BernoulliNB()` para identificar frases positivas ou negativas;
1. Teste o seu classificador com frases que você inventar.

In [ ]:
# Resolva seu exercício aqui

## Exercício 2
**Objetivo: dividir um dataset em conjuntos de treino e teste**

Até o momento, avaliamos nosso classificador de frases usando entradas manuais, e podemos ou não estar satisfeitos com ele. Porém, seria interessante termos um número que nos diz o quão efetiva foi nossa máquina no problema de classificação. É importante lembrarmos que fazer *fit* do classificador significa estimar as probabilidades relacionadas à estimação Bayesiana, e, portanto, *avaliar* o sistema significa avaliar até que ponto essas probabilidades (que foram estimadas num conjunto de dados restrito) podem extrapolar para dados nunca antes vistos pelo classificador.

Isso significa que precisamos separar nossos dados entre aqueles que serão usados para treino e aqueles que serão usados para teste. Nunca use nenhum dado do conjunto de teste para nenhum outro fim que não seja somente avaliar o resultado final do classificador!

A função `train_test_split` divide seu conjunto de dados em treinamento e teste. Uma chamada de exemplo é como abaixo:

In [30]:
X_train, y_train, X_test, y_test = train_test_split(X, y, train_size=0.6, stratify=y)

Veja a documentação da função `train_test_split` e responda:

1. O que significa dizer `train_size=0.6`?
1. O que significa dizer `stratify=y`?
1. Quais são as dimensões de `X_train`, `y_train`, `X_test` e `y_test`, e porque temos essas dimensões?

## Exercício 3
**Objetivo: entender como o `accuracy_score` funciona**

Uma das possíveis métricas de avaliação de um classificador é o accuracy score, isto é, o número de elementos classificados corretamente dividido pelo total de elementos no conjunto de teste. A biblioteca sklearn tem uma função que calcula o accuracy score.

Modifique o valor da variável `y_pred` abaixo para que o accuracy score seja igual a $0.25$.

In [31]:
y_test = np.array(['neg', 'neg', 'pos', 'pos']) # Dados corretos, retirados do conjunto de teste
y_pred = np.array(['neg', 'pos', 'neg', 'pos']) # Predições (geradas pela máquina) realizadas sobre o conjunto de teste
acc = accuracy_score(y_test, y_pred)
print(acc)

0.5


## Exercício 4
**Objetivo: treinar uma máquina de classificação e avaliá-la com accuracy score**

Agora, vamos juntar o que vimos nos exercícios anteriores e aplicar para fazer uma máquina que classifica frases entre aquelas que são "favoráveis" ao calor e aquelas que são "contrárias" ao calor. Partindo do código abaixo, implemente esse classificador. Lembre-se que:

1. Os métodos `fit` e `fit_transform` só devem ser chamados para os elementos do conjunto de treino,
1. Os elementos do conjunto de teste só devem ser parâmetros dos métodos `predict` e `transform`.

Qual foi o accuracy do seu classificador?

In [ ]:
# Dividindo dados entre treino e teste
X_train, y_train, X_test, y_test = train_test_split(X, y, train_size=0.6, stratify=y)

# Treinamento da máquina


# Coloque seu código aqui

# Teste da máquina

# Coloque seu código aqui


# Avaliação
acc = accuracy_score(y_test, y_pred)
print(acc)

## Exercício 5
**Objetivo: concentrar várias etapas do processo de classificação em uma pipeline**

Você deve ter percebido que o treino e o teste do processo de classificação executam as mesmas etapas, na mesma ordem. A biblioteca `sklearn` permite que juntemos essas duas etapas em uma única máquina através de uma `pipeline`. A pipeline é declarada usando uma lista de tuplas. Cada elemento da tupla é uma das etapas do processo de classificação, e define um nome (que pode ser qualquer coisa) e um objeto do `sklearn`. Então, os métodos `fit` e `predict` são chamados diretamente na pipeline. Por exemplo:

In [32]:
minha_pipeline = Pipeline([
                            ('meu_vetorizador', CountVectorizer()),
                            ('meu_classificador', BernoulliNB())
                            ])
minha_pipeline.fit(["olá", "mundo"], ['Classe 1', 'Classe 2'])
y_pred = minha_pipeline.predict(["olá"])
print(y_pred)

['Classe 1']


Com base nesse código, altere seu classificador para que use uma pipeline para tornar o código do exercício 4 o mais compacto que conseguir.

## Exercício 6
**Objetivo: salvar um modelo do sklearn**

Depois de treinar e avaliar nosso modelo, se gostamos dele, provavelmente vamos querer incorporá-lo em algum outro lugar. Para isso, precisaremos salvar o modelo em algum arquivo e então carregar esse arquivo no local em que vamos usar (ou: *fazer o deploy para produção*). A biblioteca para isso é a `joblib`. Joblib funciona parecido com o pickle, mas é mais eficaz para tipos numéricos. Um exemplo de uso é o seguinte:

In [33]:
minha_pipeline = Pipeline([
                            ('meu_vetorizador', CountVectorizer()),
                            ('meu_classificador', BernoulliNB())
                            ])
minha_pipeline.fit(["olá", "mundo"], ['Classe 1', 'Classe 2'])
joblib.dump(minha_pipeline, 'meu_modelo.joblib')
outra_pipeline = joblib.load('meu_modelo.joblib')
y_pred = outra_pipeline.predict(["olá"])
print(y_pred)

['Classe 1']


1. Experimente salvar seu modelo de classificação e carregá-lo em uma outra variável.
1. Qual é o tamanho do arquivo que contém o seu modelo?
1. Envie seu modelo para um colega tentar carregá-lo, e tente carregar e usar o modelo de um colega. Para isso, use o canal do Discord.

## Exercício 7
**Objetivo: corrigir um procedimento de treinamento e teste de um modelo**

Muitas vezes, encontrar um *accuracy score* elevado é entendido como o "sucesso" de um modelo. Porém, é comum termos erros nos procedimentos de treino, teste e avaliação que prejudicam o processo e podem levar a falsos resultados positivos. No código abaixo, isso acontece pelo menos uma vez. Nele, um aluno gerou um classificador de reviews de filmes e está muito contente porque o resultado foi de 100% de acerto. Corrija todos os erros do código, e explique por que esses erros atrapalham o processo de classificação.

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

df = pd.read_csv('datasets/IMDB Dataset.csv')
X = df['sentiment'].to_numpy()
y = df['review'].to_numpy()

classificador = Pipeline([
                        ('meu_vetorizador', CountVectorizer()),
                        ('meu_classificador', BernoulliNB())
                        ])
classificador.fit(X,y)
y = classificador.predict(X)
acc = accuracy_score(y,y)
print(acc)

1.0


## Exercício 8
**Objetivo: fazer e avaliar um classificador de textos**

O spam-ham dataset tem vários e-mails que são classificados como spam (mensagem não-requisitada) ou ham (mensagem normal). A ideia dessa classificação é que e-mail do tipo spam devem ser movidos para uma pasta específica.

1. Faça um sistema que recebe como entrada o texto de um e-mail e identifica se ele é spam ou ham.
1. Avalie seu sistema em relação ao accuracy score.

In [40]:
df = pd.read_csv('datasets/spam_ham_dataset.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
